In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.1'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-04-28 08:07:07--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  2.71MB/s    in 0.3s    

2022-04-28 08:07:07 (2.71 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
wirless_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [ ]:
wirless_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [ ]:
wirless_df.count()

9002021

In [ ]:
review_df = wirless_df.select(["star_rating","helpful_votes", "total_votes", "vine", "verified_purchase"])

In [ ]:
review_df.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          2|            1|          3|   N|                Y|
|          4|            0|          0|   N|                N|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          3|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|           

In [ ]:
# Filter for reviews that have a more than 20 total votes
filtered_total_votes = review_df.filter(review_df["total_votes"] >= 20)
filtered_total_votes.count()

72587

In [ ]:
# Filter for a percentage that is greater than 50,60 on helpful votes
# helpful_votes / total_votes > 0.6
percentage_filtered_total_votes = filtered_total_votes.filter(filtered_total_votes["helpful_votes"]/filtered_total_votes["total_votes"] >0.5)
percentage_filtered_total_votes.count()

65384

In [ ]:
# Filter on Vine Reviews
# Descrivbe the results that we habve using stats
vine_reviews  = percentage_filtered_total_votes.filter(percentage_filtered_total_votes["vine"] == "Y")
vine_reviews.describe().show()


+-------+------------------+-----------------+------------------+----+-----------------+
|summary|       star_rating|    helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+-----------------+------------------+----+-----------------+
|  count|               613|              613|               613| 613|              613|
|   mean|3.9086460032626427|92.26753670473083|100.45187601957586|null|             null|
| stddev| 1.088324982839408|146.5048356514182| 152.4288746080783|null|             null|
|    min|                 1|               11|                20|   Y|                N|
|    max|                 5|             1735|              1795|   Y|                Y|
+-------+------------------+-----------------+------------------+----+-----------------+



In [ ]:
# Filter on Vine Reviews
# Descrivbe the results that we habve using stats
vine_reviews  = percentage_filtered_total_votes.filter(percentage_filtered_total_votes["vine"] == "N")
vine_reviews.describe().show()

+-------+-----------------+------------------+------------------+-----+-----------------+
|summary|      star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+-----------------+------------------+------------------+-----+-----------------+
|  count|            64771|             64771|             64771|64771|            64771|
|   mean|3.610334872087817| 56.93685445646972|63.124314894011206| null|             null|
| stddev|1.612079238915757|114.63759099585798|120.58550209956607| null|             null|
|    min|                1|                11|                20|    N|                N|
|    max|                5|              5854|              6063|    N|                Y|
+-------+-----------------+------------------+------------------+-----+-----------------+



In [ ]:
# The difference between vines and non vines are that star rating mean is similar but there is a significant difference in helpful votes with vines being seen as far more helpful. 